# Command-Line Interfaces
In order to write flexible, reusable scripts, one must get information from the user and also send it back to them. Hard codeing a bunch of global constants is no way to live!

## `stdin`, `stdout` and `stderr`
These are your standard streams. The reason I started with a section on files was so I could get to these babies. They are `IO` objects that every script starts with open, and they automatically close at the end. They aren't "real" files, but they give the same interfaces as files (besides `seek`). `stdin` is open for reading and both `stdout` and `stderr` are open for writing.

As you probably know, you can send data to the `stdin` of a program by piping the output of another program to it.

```bash
$ ls / | grep "b"
bin
boot
lib
lib64
sbin
```

You can also do by using file redirection.

```bash
$ grep "b" < some_file
...
```

From inside the script, this looks like any other IO object, and you can do whatever you need with the lines.

```julia
for line in eachline(stdin)
    # do something
end
```

However, the creators of Julia know that this is such a common case that both `readlines` and `eachline` default to using stdin. `eachline()` is identical to `eachline(stdin)`

`stdout` is the easy one. You already know how to write to it: the `print` and `println` functions. You can also use `write`, of course, if you need to write binary data.

`stderr` is exactly the same as stdout, but you'd explicitely state that you wanted things to go there: 

In [1]:
println(stderr, "things are real bad in this script")

things are real bad in this script


Normally, you want to send data to stdout that is suitable to be used by the stdin of another program (maybe `grep` or `sed`?), and `stderr` is for messages for the user about what's happening in the script (error messages, logging, debugging info). For more advanced logging, Julia provides a [Logging module](https://docs.julialang.org/en/v1/stdlib/Logging/) in the standard library.

## CLI Arguments

Another important way to get information for your uses is through command line arguments. As in most languages, you get an array of strings. Unlike many languages, the first item in this array is _not_ the name of the program. That's in a global variable called `PROGRAM_FILE`. That can also be useful, but we're just talking about the `ARGS` array for now.

Here is a simple clone of `cp`:

```julia
# cp.jl

function main()
    dest = ARGS[end]
    srcfiles = ARGS[1:end-1]
    
    if isdir(dest)
        dest = joinpath.(dest, basename.(srcfiles))
    end

    cp.(srcfiles, dest)
end

main()
```

Which you would use like this :

```bash
$ julia cp.jl afile otherfile targetdir
```

We don't really need the main function here, it's just best practice to put everything besides constants inside of a function in Julia for performance reasons (globals are slow unless they are constants), and because it leads to more modular, reusable code.

For more sophisticated argument parsing, two popular third-party modules are [ArgParse.jl](https://juliaobserver.com/packages/ArgParse) and [DocOpt.jl](https://juliaobserver.com/packages/DocOpt), which provide similar interfaces to the Python modules of the same names.

> _Note on vectorized functions_:
>
> If you're new to Julia, you might have trouble understanding a couple of lines:
>
>     dest = joinpath.(dest, basename.(srcfiles))
>
> and
>
>     cp.(srcfiles, dest)
>
> These lines make use of the Julia's [dot syntax for vectorizing functions](https://docs.julialang.org/en/v1/manual/functions/#man-vectorized-1) as an alternative to loops. In the first case, `srcfiles` is a vector of strings. `basename.(srcfiles)` returns an array of the basename of each path in `srcfiles`. It's the same as `[basename(s) for s in srcfiles]`. Each element in this array is then joined with the original `dest` directory for the full path. Because this operation contains nested dot operations, they are all _fused_ into a single loop for greater efficiency.
>
> Because `dest` can now either be a vector or a string, `cp.(srcfiles, dest)` can mean two different things: If `dest` is still a string, something like this happens:
>
> ```julia
> for file in srcfiles
>     cp(file, dest)
> end
> ```
>
> If dest has become a vector, however, it means this:
> ```julia
> for (file, target) in zip(srcfiles, dest)
>     cp(file, target)
> end
> ```
>
> This is handy for our case because, no matter which type `dest` has in the end, the vectorized version will do the right thing!
>
> For more on the nuances of vectorizing functions, check out the documentation on [broadcasting](https://docs.julialang.org/en/v1/manual/arrays/#Broadcasting-1)


## Environment Variables and Config Files

Another way to get info from your user is from configuration settings. Though it is not the approach I prefer, one popular way to do this is using environment variables store settings, which are exported in `~/.profile` or some other shell configuration file. In Julia, environment variables are stored in the `ENV` dictionary.

In [2]:
for var in ("SHELL", "EDITOR", "USER")
    @show var ENV[var]
end

var = "SHELL"
ENV[var] = "/usr/bin/zsh"
var = "EDITOR"
ENV[var] = "nvim"
var = "USER"
ENV[var] = "ninjaaron"


I personally prefer to use config files. [TOML](https://github.com/toml-lang/toml) seems to be what all the cool kids are using these days, and it's also used by Julia's built-in package manager, so that's probably not a bad choice. There is a "secret" TOML module in the standard library which is vendor by `Pkg`.

You can get at it this way:

In [3]:
import Pkg: TOML
@doc TOML.parse

Executes the parser, parsing the string contained within.

This function will return the `Table` instance if parsing is successful, or it will return `nothing` if any parse error or invalid TOML error occurs.

If an error occurs, the `errors` field of this parser can be consulted to determine the cause of the parse failure.

Parse IO input and return result as dictionary.

Parse string


Because it's vendored, it's probably considered an implementation detail and subject to disappaer without notice. I don't know what the deal is. Anyway, the library they vendor can be found [here](https://github.com/wildart/TOML.jl). There are a couple other TOML libraries on juliaobserver.com. There are also a semi-official looking packages under the JuliaIO org on github called [ConfigParser.jl](https://github.com/JuliaIO/ConfParser.jl) That can deal with ini files a few other types. There is also a [JSON.jl](https://github.com/JuliaIO/JSON.jl). I'm pretty against using JSON for config files, but there it is.